# HYPERPARAMETER OPTIMIZATION USING OPTUNA

## IMPORTS

In [1]:
##################
# IMPORT MODULES #
##################
# SYS IMPORT
import os, inspect, importlib, argparse
import gc
import pandas as pd
import numpy as np
from pathlib import Path

from tqdm import tqdm
import matplotlib.pyplot as plt

import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
import optuna 
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## CONFIG

In [2]:
config = {
    "DATA_PATH" : "D:/Documents/GitHub/ml-pipeline/data/TPS-APR2021/train.csv",
    "TARGET_VAR" : "Survived"
}

## LOADING DATA

In [3]:
df = pd.read_csv(config["DATA_PATH"])
target = df[config["TARGET_VAR"]]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


## FEATURE ENGINEERING

In [4]:
import datetime
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import is_integer_dtype, is_float_dtype, is_object_dtype

def missing_values(dataframe):
    # Droping features that have too many MV
    drop_features = ["Cabin", "Name", "Ticket"]
    dataframe = dataframe.drop(drop_features, axis=1)
    # DIVIDE INTO NUMERICAL FEATURES AND CATEGORICAL FEATURES
    integer_features = [col for col in dataframe.columns if is_integer_dtype(dataframe[col])]
    float_features = [col for col in dataframe.columns if is_float_dtype(dataframe[col])]
    object_features = [col for col in dataframe.columns if is_object_dtype(dataframe[col])]

    # WE REPLACE MISSING VALUES IN INTEGER  & FLOAT FEATURES WITH MEAN AND MODE FOR CATEGORICAL FEATURES
    dataframe[integer_features] = dataframe[integer_features].apply(lambda x: x.fillna(value=x.mean().astype(int)))
    dataframe[float_features] = dataframe[float_features].apply(lambda x: x.fillna(value=x.mean()))
    dataframe[object_features] = dataframe[object_features].apply(lambda x: x.fillna(value=x.mode()[0]))

    # ASSERT WE DON'T HAVE ANY MISSING VALUES IN THE DATASET
    assert dataframe.columns[dataframe.isnull().any()].empty, 'We still have some missing values in the dataset!'
    return dataframe

def get_cat_features(dataframe):
    categorical_features = [col for col in dataframe.columns if is_object_dtype(dataframe[col])]
    return categorical_features

def cat_encoding(dataframe, features):
    le = LabelEncoder()
    dataframe[features] = dataframe[features].apply(lambda x: le.fit_transform(x))
    return dataframe

def feature_engineering(dataframe, train=True):
    # FEATURE ENG
    dataframe = missing_values(dataframe)
    features_cat = get_cat_features(dataframe)
    dataframe = cat_encoding(dataframe, features_cat)
    features = dataframe.columns.difference(["Survived", "PassengerId", "kfold"])
    # RETURN DATAFRAME & ALL FEATURES NEEDED FOR TRAINING OR PREDICTION
    return dataframe, features

In [5]:
df, features = feature_engineering(df)

## OPTIMIZING

In [6]:
def objective(trial, data=df[features], target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=95)
    param = {
        'loss_function' : "CrossEntropy",
        'custom_metric': 'Accuracy', 
        'random_state': 95,
        'n_estimators': 10000,
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bylevel': trial.suggest_categorical('colsample_bytree', [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [2,4,8,16]),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),

    }
    model = ctb.CatBoostClassifier(**param)  
    
    model.fit(train_x,train_y, eval_set=[(test_x,test_y)], early_stopping_rounds=100, verbose=100)
    
    preds = model.predict(test_x)
    
    acc = accuracy_score(test_y, preds)
    
    return acc

In [7]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=3600)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

4885157 (4800)	total: 40.2s	remaining: 43.6s
4900:	learn: 0.4852331	test: 0.4884832	best: 0.4884831 (4899)	total: 41.1s	remaining: 42.7s
5000:	learn: 0.4851760	test: 0.4884378	best: 0.4884378 (5000)	total: 42s	remaining: 42s
5100:	learn: 0.4851231	test: 0.4883972	best: 0.4883972 (5100)	total: 42.6s	remaining: 41s
5200:	learn: 0.4850708	test: 0.4883565	best: 0.4883562 (5199)	total: 43.3s	remaining: 40s
5300:	learn: 0.4850170	test: 0.4883094	best: 0.4883094 (5300)	total: 44s	remaining: 39s
5400:	learn: 0.4849675	test: 0.4882748	best: 0.4882742 (5392)	total: 44.8s	remaining: 38.2s
5500:	learn: 0.4849207	test: 0.4882426	best: 0.4882425 (5499)	total: 45.6s	remaining: 37.3s
5600:	learn: 0.4848734	test: 0.4882085	best: 0.4882085 (5598)	total: 46.2s	remaining: 36.3s
5700:	learn: 0.4848299	test: 0.4881777	best: 0.4881777 (5700)	total: 46.9s	remaining: 35.4s
5800:	learn: 0.4847856	test: 0.4881480	best: 0.4881480 (5800)	total: 47.5s	remaining: 34.4s
5900:	learn: 0.4847448	test: 0.4881163	best: 0.

## VISUALIZATION

In [11]:
optuna.visualization.plot_optimization_history(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [12]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [14]:
params=study.best_params
params

{'reg_alpha': 0.023013164688329528,
 'reg_lambda': 0.003811720979048805,
 'colsample_bytree': 0.3,
 'subsample': 0.6,
 'learning_rate': 0.02,
 'max_depth': 100,
 'num_leaves': 186,
 'min_child_samples': 225,
 'min_data_per_groups': 74}